In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP
from cornac.models import *

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


/Users/manyi/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open ('/Users/manyi/Documents/Projects/Cornac/Algorithms/cdl/mind_algorithm_test.csv') as mind_file:
    mind = pd.read_csv(mind_file)
mind['story'] = mind['story'].astype('int')
mind_ = mind[:10000]

In [3]:
# pre-process data to initialize ctr model
mind_feedback = mind_.loc[:, ['user_id','item_id','rating']]
# feedback = cornac.data.Dataset.from_uir(mind_feedback.itertuples(index=False))
feedback = mind_feedback.apply(lambda x: tuple(x), axis=1).values.tolist()
text = list(mind_['text'])
item_ids = list(mind_['item_id'])
# Instantiate a TextModality, it makes it convenient to work with text auxiliary information
item_text_modality = TextModality(
    corpus=text,
    ids=item_ids,
    tokenizer=BaseTokenizer(sep=" ", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)

# Define an evaluation method to split feedback into train and test sets
mind_ratio_split = RatioSplit(
    data=feedback,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=0.5,
)

rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 181
Number of items = 3954
Number of ratings = 7923
Max rating = 1.0
Min rating = 0.0
Global mean = 0.7
---
Test data:
Number of users = 169
Number of items = 935
Number of ratings = 1364
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 181
Total items = 3954


/Users/manyi/Library/Python/3.9/lib/python/site-packages/cornac/data/dataset.py:361: UserWarning: 77 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/Users/manyi/Library/Python/3.9/lib/python/site-packages/cornac/data/dataset.py:361: UserWarning: 3 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [4]:
import diversity_mind
def compute_avg_diversity(model, data, feature):
    if (feature=="category"):
        for i in range(len(set(model.train_set.user_ids))): # get the average result for all users from training data
            result = []
            r = diversity_mind.Retrieval(model=model,data = data,UIDX = i,TOPK = 50,feature = feature)
            d = diversity_mind.Diversity(r.get_history(), r.get_recy())
            result.append(d.compute())
    else:
        useridx = list(set([int(tuple[1]) for tuple in model.train_set.uid_map]))
        result = []
        i = 0
        while i <= 200:
            # shuffle
            user_pair = random.sample(useridx, 2)
            r0 = diversity_mind.Retrieval(model=model, data=data, UIDX=user_pair[0],TOPK = 50, feature=feature)
            r1 = diversity_mind.Retrieval(model=model, data=data, UIDX=user_pair[1],TOPK = 50, feature=feature)
            d = diversity_mind.Diversity(r0.get_recy(), r1.get_recy())
            result.append(d.compute())
            i += 1
    return np.mean(result)

In [5]:
import os
import glob

folder_path = os.getcwd()

In [6]:
all_data = []
for k in [10,50,100,200,300,400]:
    for v in [1,2,5,10]:
        cdl = CDL(k=k, autoencoder_structure=[200], max_iter=10, lambda_u=0.1, lambda_v=1, lambda_w=0.1, lambda_n=1000, seed=123)
        metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
                Recall(k=50), NDCG(k=50), NCRR(k=50),
                MRR(),AUC(), MAP()]

        # Put everything together into an experiment and run it
        cornac.Experiment(
            eval_method=mind_ratio_split, models=[cdl], metrics=metrics, user_based=False
        ).run()

        pattern = os.path.join(folder_path, "CornacExp*")

        files = glob.glob(pattern)

        files.sort(key=os.path.getmtime)

        latest_file_path = files[-1]

        with open(latest_file_path, "r") as file:
            lines = file.readlines()
            if len(lines) >= 6:
                sixth_line = lines[5]
                print(sixth_line)
                string_without_spaces = " ".join(sixth_line.split())
                split_string = string_without_spaces.split("|")
                print(split_string)

        
        data = {"model":"CDL",
        "data":"mind",
        "k":k,
        "max_iter":10,
        "lambda_v":0.01,
        "l2_reg":0.001,
        "MAE":split_string[1],"MSE":split_string[2],"RMSE":split_string[3],"AUC":split_string[4],
        "F1":split_string[5],"MAP":split_string[6],"MRR":split_string[7],"NCRR":split_string[8],
        "NDCG":split_string[9],"Precision":split_string[10],"Recall":split_string[11],
        "category diversity":compute_avg_diversity(cdl, mind_, "category"),
        "story diversity":compute_avg_diversity(cdl, mind_, "story")}
        
        print(data["category diversity"])
        print(data["story diversity"])

        all_data.append(data)
    



[CDL] Training started!


100%|██████████| 10/10 [00:09<00:00,  1.05it/s, loss=25] 


Learning completed!

[CDL] Evaluation started!


Ranking: 100%|██████████| 169/169 [00:00<00:00, 744.01it/s]



TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CDL | 0.6047 | 0.5918 | 0.7693 | 0.5024 | 0.0033 | 0.0041 | 0.0098 |  0.0033 |  0.0078 |       0.0019 |    0.0197 |   12.0218 |   0.2413

CDL | 0.6047 | 0.5918 | 0.7693 | 0.5024 | 0.0033 | 0.0041 | 0.0098 |  0.0033 |  0.0078 |       0.0019 |    0.0197 |   12.0218 |   0.2413

['CDL ', ' 0.6047 ', ' 0.5918 ', ' 0.7693 ', ' 0.5024 ', ' 0.0033 ', ' 0.0041 ', ' 0.0098 ', ' 0.0033 ', ' 0.0078 ', ' 0.0019 ', ' 0.0197 ', ' 12.0218 ', ' 0.2413']
0.30902791647326533
1.0259482385889656

[CDL] Training started!


...


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CDL | 0.5206 | 0.4473 | 0.6688 | 0.7125 | 0.0103 | 0.0099 | 0.0230 |  0.0079 |  0.0243 |       0.0058 |    0.0750 |   10.8041 |   0.3389

CDL | 0.5206 | 0.4473 | 0.6688 | 0.7125 | 0.0103 | 0.0099 | 0.0230 |  0.0079 |  0.0243 |       0.0058 |    0.0750 |   10.8041 |   0.3389

['CDL ', ' 0.5206 ', ' 0.4473 ', ' 0.6688 ', ' 0.7125 ', ' 0.0103 ', ' 0.0099 ', ' 0.0230 ', ' 0.0079 ', ' 0.0243 ', ' 0.0058 ', ' 0.0750 ', ' 10.8041 ', ' 0.3389']
0.2709421509237354
1.2310263905007373

[CDL] Training started!


100%|██████████| 10/10 [00:10<00:00,  1.09s/it, loss=23] 


Learning completed!

[CDL] Evaluation started!


Ranking: 100%|██████████| 169/169 [00:00<00:00, 515.64it/s]



TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CDL | 0.5206 | 0.4473 | 0.6688 | 0.7125 | 0.0103 | 0.0099 | 0.0230 |  0.0079 |  0.0243 |       0.0058 |    0.0750 |   11.3482 |   0.3415

CDL | 0.5206 | 0.4473 | 0.6688 | 0.7125 | 0.0103 | 0.0099 | 0.0230 |  0.0079 |  0.0243 |       0.0058 |    0.0750 |   11.3482 |   0.3415

['CDL ', ' 0.5206 ', ' 0.4473 ', ' 0.6688 ', ' 0.7125 ', ' 0.0103 ', ' 0.0099 ', ' 0.0230 ', ' 0.0079 ', ' 0.0243 ', ' 0.0058 ', ' 0.0750 ', ' 11.3482 ', ' 0.3415']
0.2709421509237354
1.2447362892290004


In [7]:
df = pd.DataFrame(columns = all_data[0].keys())
i=0
for d in all_data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [8]:
df

,model,data,k,max_iter,lambda_v,l2_reg,MAE,MSE,RMSE,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity,story diversity
0,CDL,mind,10,10,0.01,0.001,0.6047,0.5918,0.7693,0.5024,0.0033,0.0041,0.0098,0.0033,0.0078,0.0019,0.0197,0.309028,1.025948
1,CDL,mind,10,10,0.01,0.001,0.6047,0.5918,0.7693,0.5024,0.0033,0.0041,0.0098,0.0033,0.0078,0.0019,0.0197,0.309028,1.053029
2,CDL,mind,10,10,0.01,0.001,0.6047,0.5918,0.7693,0.5024,0.0033,0.0041,0.0098,0.0033,0.0078,0.0019,0.0197,0.309028,1.026917
3,CDL,mind,10,10,0.01,0.001,0.6047,0.5918,0.7693,0.5024,0.0033,0.0041,0.0098,0.0033,0.0078,0.0019,0.0197,0.309028,1.112629
4,CDL,mind,50,10,0.01,0.001,0.5739,0.5344,0.7310,0.5880,0.0070,0.0063,0.0153,0.0057,0.0153,0.0040,0.0392,0.271277,1.349881
5,CDL,mind,50,10,0.01,0.001,0.5739,0.5344,0.7310,0.5880,0.0070,0.0063,0.0153,0.0057,0.0153,0.0040,0.0392,0.271277,1.329184
6,CDL,mind,50,10,0.01,0.001,0.5739,0.5344,0.7310,0.5880,0.0070,0.0063,0.0153,0.0057,0.0153,0.0040,0.0392,0.271277,1.335924
7,CDL,mind,50,10,0.01,0.001,0.5739,0.5344,0.7310,0.5880,0.0070,0.0063,0.0153,0.0057,0.0153,0.0040,0.0392,0.271277,1.239493
8,CDL,mind,100,10,0.01,0.001,0.5490,0.4921,0.7015,0.6146,0.0061,0.0066,0.0146,0.0046,0.0125,0.0035,0.0306,0.487083,1.160003
9,CDL,mind,100,10,0.01,0.001,0.5490,0.4921,0.7015,0.6146,0.0061,0.0066,0.0146,0.0046,0.0125,0.0035,0.0306,0.487083,1.155492


In [10]:
df.to_csv('/Users/manyi/Documents/Projects/Cornac/data/cdl_mind_data.csv',header=-1,index=0)